# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_with_intrinsic_reward_V2 import * #PPO_Agent_v2 PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-03-06 14:57:45.292859: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 34077


2024-03-06 14:57:46.706815: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:34077/ (Press CTRL+C to quit)


In [2]:
ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<Continuous_MountainCarEnv<MountainCarContinuous-v0>>>>

In [4]:
observation = env.reset()

# Initialize an empty list to store rewards
rewards = []

# Run episodes and collect rewards
for episode in range(10000):
    done = False
    while not done:
        observation, reward, done, info = env.step(env.action_space.sample())
        rewards.append(reward)

# Calculate normalization factor (e.g., mean or standard deviation)
mean_reward = np.mean(rewards)
std_dev_reward = np.std(rewards)
mean_reward, std_dev_reward

(46.85646446271359, 49.903434326465344)

In [5]:
s = env.observation_space.sample()
s

array([-0.36262614,  0.04793081], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), 2)

In [7]:
s = env.reset()
s[0]

-0.43041316

In [8]:
env.action_space

Box(-1.0, 1.0, (1,), float32)

### Training/Hyperparam run

In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = False
writer= "logs_hyper/fit_PPO_intrinsic_reward/"

In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_intrinsic_reward"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_intrinsic_reward/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.99,
                gae_min = 0.90, gae_max = 0.96, policy_clip =0.2,
                lr_actor_min = 0.00001, lr_actor_max = 0.001,
                lr_critic_min = 0.00001, lr_critic_max = 0.001,
                entropy_factor_min = 0.01, entropy_factor_max = 0.5,
                dense_min = 32, dense_max = 256,
                environment_name=ENV, num_layers_act = 2, num_layers_crit =2, num_layers_model = 1, training_epoch = 100,
                memory_size = env._max_episode_steps, normalize=False,
                scaling_factor_reward = 0.1
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        training_steps = 2000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_actor = [256]#64, 32]
        num_layers_actor = 1
        dense_units_critic = [256]#64,32]
        num_layers_crit =1
        num_layer_m = 1
        dense_units_model = [128]

        model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, dense_units_model,  num_layers_actor, num_layers_crit, num_layer_m, writer, 
                      environment_name = ENV, return_agent = True, lr_actor= 0.0001, lr_critic= 0.0001, sucess_criteria_epochs=SUCESS_CRITERIA_EPOCH, sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 20, scaling_factor_reward = 0.1, lr_model= 0.00001,
                      memory_size = env._max_episode_steps)


Acquiring parameters ....
Trial number :  0


  5%|▌         | 102058/2000000 [06:59<6:20:46, 83.07it/s] 

Epoch: 102000 : Reward Train: -0.25916866437752034 


  5%|▌         | 104047/2000000 [07:07<6:32:50, 80.44it/s] 

Epoch: 104000 : Reward Train: -0.25918177904240974 


...

 13%|█▎        | 268055/2000000 [18:28<5:56:16, 81.02it/s] 

Epoch: 268000 : Reward Train: -0.04450406290896061 


 14%|█▎        | 270048/2000000 [18:36<5:50:22, 82.29it/s] 

Epoch: 270000 : Reward Train: -0.0445080331111384 


 14%|█▎        | 270716/2000000 [18:38<1:32:53, 310.28it/s]

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=5):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"
for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
    training_steps = 500000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    gae = trials.hyperparameters.values["gae_lambda"]
    policy_clip = trials.hyperparameters.values["policy_clip"]
    scaling_factor_reward= trials.hyperparameters.values["scaling_factor_reward"]
    
    
    lr_actor=  trials.hyperparameters.values["lr_actor"]
    lr_critic=  trials.hyperparameters.values["lr_critic"]
    lr_model=  trials.hyperparameters.values["lr_model"]
    
    try:
        n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
    except : 
        n_dense_layers_actor = 1
        
    try:
        n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
    except:
        n_dense_layers_critic = 1
        

    dense_layers_actor = []
    for i in range(n_dense_layers_actor):
        dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

    dense_layers_critic = []
    for i in range(n_dense_layers_critic):
        dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

    
    n_dense_layers_model = 1
    dense_layers_model = []
    for i in range(n_dense_layers_model):
        dense_layers_model.append(trials.hyperparameters.values['n_dense_layers_model'+str(i)])

    model = run_training(
        training_steps = training_steps,   
            discount = discount,
            dense_units_act = dense_layers_actor, 
            dense_units_crit = dense_layers_critic,
            dense_units_model = dense_layers_model,
            num_layer_a = n_dense_layers_actor,
            num_layer_c = n_dense_layers_critic,
            num_layer_m = n_dense_layers_model,
            writer = writer,  
            save_factor=50000, 
            sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
            sucess_criteria_value = SUCESS_CRITERIA_VALUE,
            environment_name=ENV,
            evaluation_epoch = env._max_episode_steps,
            return_agent = True,
            lr_actor= lr_actor, 
            lr_critic= lr_critic,
            lr_model= lr_model,
            gae_lambda=gae,
            training_epoch= 200,
            entropy_coeff= entropy_factor,
            policy_clip = policy_clip,
            memory_size= env._max_episode_steps,
            id = int(trials.trial_id),
            scaling_factor_reward = scaling_factor_reward)
        
    break
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_intrinsic_"+exploration_tech+"_video.mp4", env= ENV)